In [40]:
import pandas as pd

In [41]:
rationales = pd.read_excel('/home/huy/Desktop/HCMUS/distilling-step-by-step/[API] RATIONALES/Contrastive (1).xlsx', index_col=0)

In [42]:
rationales.rationales[15].lower().split('most likely answer: ')

['6. the most likely answer is (e) gather flowers because if a child is in a meadow in spring, they might engage in activities like picking or gathering flowers. the other choices (clean room, set table, play with toys, form opinions) are either unrelated or not specific to being in a meadow.']

In [43]:
rationales['split'] = rationales.rationales.apply(lambda x: x.lower().split('most likely answer'))

In [44]:
rationales.split = rationales.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [45]:
import ast

# convert choices to list
rationales['choices'] = rationales.choices.apply(lambda x: ast.literal_eval(x))

In [46]:
# get the index of the correct answer
import numpy as np
rationales['correct_index'] = [{} for _ in range(len(rationales))]
for index, row in rationales.iterrows():
    for choice in row['choices']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].index(choice)

In [47]:
# get the key of value min
rationales['LLM_answer'] =rationales.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [48]:
# HANDLE EXCEPTION
error = rationales[rationales.LLM_answer.isna()]
error.head(5)

,question,choices,answer,rationales,split,correct_index,LLM_answer
182,After the guilty verdict in the killing the ju...,"[wrong, death of, bad, negative, evil]",evil,"3. After the guilty verdict in the killing, th...",because it reflects the severe judgment and c...,{},NaN
183,After the guy is successful in cashing in his ...,"[sad, quitting, extra money, great joy, leave]",great joy,4. After the guy is successful in cashing in h...,because it is a common emotional response to ...,{},NaN
185,"After the lunch break, what did the men resume...","[working, fix it, accelerate, putting together...",working,"6. After the lunch break, the men resumed (A) ...",because it is a common activity one would ret...,{},NaN
278,"Bill was awaking, what was happening to him ph...","[tiredness, grogginess, discomfort, getting ou...",eyes opening,9. (E) eyes opening: The question asks what is...,is that his eyes are opening.,{},NaN
340,"By laughing while playing, what were the child...","[learning, getting tired, injuries, having fun...",having fun,1. (D) having fun: The fact that the children ...,because it directly aligns with the behavior ...,{},NaN


In [49]:
# get the index of the correct answer
import numpy as np
error['correct_index'] = [{} for _ in range(len(error))]
for index, row in error.iterrows():
    for choice in row['choices']:
        if choice in row['rationales']:
            row['correct_index'][choice] = row['rationales'].index(choice)

<ipython-input-49-25441d68bc9a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['correct_index'] = [{} for _ in range(len(error))]


In [50]:
# get the key of value min
error['LLM_answer'] = error.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

<ipython-input-50-1f6358438110>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['LLM_answer'] = error.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)


In [51]:
print(error[error.LLM_answer.isna()].question)

Series([], Name: question, dtype: object)


In [52]:
rationales.LLM_answer = rationales.LLM_answer.fillna(error.LLM_answer)

In [53]:
(sum(rationales.LLM_answer == rationales.answer)) / len(rationales)

0.7416529830322933

In [54]:
main = pd.read_csv('/home/huy/Desktop/HCMUS/total.csv', index_col=0)

In [55]:
rationales

,question,choices,answer,rationales,split,correct_index,LLM_answer
0,"""There are 10 apples on an apple tree. Three ...","[park, coloring book, garden center, math prob...",math problem,1.\nMost likely answer: (D) math problem\nThis...,: (d) math problem\nthis is the,{'math problem': 6},math problem
1,"A John is a bum. Much like the stereotype, he...","[bus depot, beach, train station, bridge, bridge]",bridge,2.\nMost likely answer: (A) bus depot\nThis is...,: (a) bus depot\nthis is the,{'bus depot': 6},bus depot
2,A bad person places little value on being hone...,"[excellent, upright, premium, competent, sincere]",sincere,3.\nMost likely answer: (B) upright\nThis is t...,: (b) upright\nthis is the,{'upright': 6},upright
3,"A bald eagle flies over St. Paul, where is it?","[texas, thermal, minnesota, canada, photograph]",minnesota,4. \nMost likely answer: (C) minnesota\nThis i...,: (c) minnesota\nthis is the,{'minnesota': 6},minnesota
4,A battleship is a powerful vessel. If you nee...,"[yatch, corvette, aircraft carrier, destroyer,...",corvette,5. \nMost likely answer: (D) destroyer\nThis i...,: (d) destroyer\nthis is the,{'destroyer': 6},destroyer
...,...,...,...,...,...,...,...
10957,what does someone have that causes them commit...,"[problems, distress, fear, go to jail, killer]",problems,8. The most likely answer is (E) killer. This ...,is (e) killer. this is because someone who co...,{'killer': 8},killer
10958,what is printed with ink and distributed daily?,"[fountain pen, squid, newspaper, book, printer]",newspaper,9. The most likely answer is (C) newspaper. Th...,is (c) newspaper. this is because newspapers ...,{'newspaper': 8},newspaper
10959,when communicating with my boss what should i do,"[misunderstandings, transfer of information, l...",transfer of information,10. The most likely answer is (B) transfer of ...,is (b) transfer of information. this is becau...,{'transfer of information': 8},transfer of information
10960,where is a good place to obtain new soap?,"[supermarket, washing, cabinet, own home, sink]",supermarket,The most likely answer would be (A) supermarke...,would be (a) supermarket. this is because sup...,"{'supermarket': 14, 'washing': 290, 'cabinet':...",supermarket


In [56]:
main['question'] = main.input.apply(lambda x: x.split('\n')[0])
a = rationales.set_index('question')
b = main.set_index('question')

In [63]:
sum(b.loc[a.index]['llm_label'].values == a.answer.values) / len(b)

0.7457580733442802

In [66]:
rationales.to_csv('/home/huy/Desktop/HCMUS/distilling-step-by-step/[API] RATIONALES/Contrastive - full.csv')